### Folium Interactive Map

The Folium interactive maps experienced heavy loading times due to the full dataset being embedded in the HTML. By offloading data storage and distribution to a dedicated backend, the map now loads only the necessary data dynamically, greatly enhancing performance and scalability.

#### Backend Architecture 


To improve performance and scalability, the following components were integrated into the new backend solution:

- **Cloud Server**: Hosts the entire backend stack.

- **Docker**: Containerization platform for managing services.

- **PostGIS**: PostgreSQL extension used for spatial data storage.

- **GDAL** (ogr2ogr): Used to ingest and upload geospatial data into the PostGIS database.

- **GeoServer**: Distributes data using OGC standards (WMS, WFS) to the Folium map.

In [ ]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
OUTPUT_DIRECTORY = NOTEBOOK_PATH / "docs"

#### Toumpa

##### Create the folium map, add basemaps and custom legend

In [ ]:
import folium
import folium.features
from folium import JsCode
from folium.plugins import MarkerCluster, Fullscreen, Realtime
from custom_control import CustomControl

# Create a folium map with no default basemap
m = folium.Map(
    location=[40.618, 22.975],
    zoom_start=14,
    tiles=None  # We'll add our own basemap below
)

# Add the CartoDB Positron tile layer (default on)
folium.TileLayer(
    'CartoDB positron',
    name='CartoDB Positron',
    control=True,
    show=True  # Show this layer by default
).add_to(m)

# Add the OpenStreetMap tile layer (off by default)
folium.TileLayer(
    'OpenStreetMap',
    name='OpenStreetMap',
    control=True,
    show=False  # Do not show this layer by default
).add_to(m)

# Load your custom legend HTML from the file
with open("my_legend.html", "r", encoding="utf-8") as f:
    legend_html = f.read()

# Create a custom control for the legend and add it to the map at the bottom right
legend_control = CustomControl(legend_html, position='bottomright')
m.add_child(legend_control) 

##### Add the WMS layers

In [ ]:
# Buildings WMS layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",  # workspace-level endpoint
    name="Buildings Toumpa (WMS)",
    layers="project1:buildings_toumpa",  # exactly as it appears in GetCapabilities
    styles="",                           # empty if using the default style
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, # zoom level where the wms layer is visible
).add_to(m)

# Toumpa WMS area layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",
    name="Area of Toumpa (WMS)",
    layers="project1:area_toumpa",
    styles="",                           
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, 
).add_to(m)

# Walking Network WMS area layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",
    name="Walking network (WMS)",
    layers="project1:walking_network_toumpa", 
    styles="",                           
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, 
).add_to(m)

##### Add the WFS layer for bus stops Markers

In [ ]:
# WFS layer for bus stops
wfs_bust = (
    "https://thanosgis.me/geoserver/wfs?"
    "service=WFS&version=2.0.0&request=GetFeature&"
    "typeName=project1:bus_stops_toumpa&srsName=EPSG:4326&outputFormat=application/json" #specify the crs "srsName=EPSG:4326"
)

# Install those plugins to use Awesome markers
folium.Element(
    '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.awesome-markers/2.0.4/leaflet.awesome-markers.css">'
).add_to(m)
folium.Element(
    '<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.awesome-markers/2.0.4/leaflet.awesome-markers.js"></script>'
).add_to(m)
folium.Element(
    '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">'
).add_to(m)

# Create a FeatureGroup for bus stops:
bus_stops_fg = folium.FeatureGroup(name="Bus stops (WFS)").add_to(m)

# Create a MarkerCluster and add it to that FeatureGroup:
cluster = MarkerCluster().add_to(bus_stops_fg)

# Custom Javascript for point_to_layer
point_to_layer_js = JsCode("""
function(feature, latlng) {
    // Create a marker using a Font Awesome icon via AwesomeMarkers
    var marker = L.marker(latlng, {
        icon: L.AwesomeMarkers.icon({
            icon: 'bus',        // Name of the Font Awesome icon
            prefix: 'fa',       // Font Awesome prefix
            markerColor: 'red' // Marker color (options like 'blue', 'red', etc.)
        })
    });
    // Bind a popup if the feature has a 'name' property
    if (feature.properties && feature.properties.name) {
        marker.bindPopup("<strong>Bus Stop:</strong> " + feature.properties.name);
    }
    return marker;
}
""")

# Realtime for bus stops
rt1 = Realtime(
    wfs_bust,
    get_feature_id=JsCode("(f) => { return f.properties.stop_id; }"),
    #interval=10000,  # update every 10 seconds
    container=cluster,  # Add to the cluster
    point_to_layer=point_to_layer_js
)

# Add the FeatureGroup to the map
bus_stops_fg.add_child(rt1)

##### Add the WFS layer for the buildings Attributes

In [ ]:
wfs_buildingst = (
    "https://thanosgis.me/geoserver/wfs?"
    "service=WFS&version=2.0.0&request=GetFeature&"
    "typeName=project1:buildings_toumpa&srsName=EPSG:4326&outputFormat=application/json" #specify the crs "srsName=EPSG:4326"
)

# Custom javascript for tooltip
tooltip_js = JsCode("""
function onEachFeature(feature, layer) {
    if (feature.properties) {
        // Build an HTML string with inline CSS
        var content = '<div style="font-size:12px; color:#333;">' +
                      '<strong>Building ID:</strong> ' + feature.properties.building_id + '<br>' +
                      '<strong>Nearest Bus Stop:</strong> ' + feature.properties.bus_stop + '<br>' +
                      '<strong>Walking Distance (m):</strong> ' + feature.properties.walking_distance_m + '<br>' +
                      '<strong>Walking Time (min):</strong> ' + feature.properties.walking_time_min +
                      '</div>';

        layer.bindTooltip(content); // bindPopup for popup
    }
}
""")

# Custom javascript for style
style_js  = JsCode("""
function(feature) {
    return {
        color: 'transparent',     // No polygon outline
        fillColor: 'transparent', // No polygon fill
        weight: 0,               // Zero line weight
        fillOpacity: 0           // Fully transparent fill
    };
}
""")

# Create the Realtime layer to poll the WFS endpoint
rt = Realtime(
    wfs_buildingst,
    get_feature_id=JsCode("(f) => { return f.properties.building_id; }"),
    style=style_js ,      # Apply the transparent style
    on_each_feature=tooltip_js,   # Bind the tooltip
    #interval=10000,                      
)

# Do not add Realtime directly to the map because it adds an extra layer on the layer control
fg = folium.FeatureGroup(name="Attributes (WFS)").add_to(m)
fg.add_child(rt)

##### Display the map

In [ ]:
# Add a fullscreen button
Fullscreen().add_to(m)

# Add a layer control so we can toggle layers on/off
folium.LayerControl().add_to(m)

# Display map
m

#### Ano poli

In [ ]:
# Create a folium map
m2 = folium.Map(
    location = (40.640978,22.9534333),
    zoom_start = 15,
    control_scale = True,
    tiles=None
)

# Add the base tile layers first
carto_map = folium.TileLayer(
    'CartoDB positron',
    name='CartoDB Positron',
    control=True,
    show=True  # This one shows by default
).add_to(m2)

open_street_map = folium.TileLayer(
    'OpenStreetMap',
    name='OpenStreetMap',
    control=True,
    show=False  # Off by default
).add_to(m2)

# Load your custom legend HTML from the file
with open("my_legend.html", "r", encoding="utf-8") as f:
    legend_html = f.read()

# Create a custom control for the legend and add it to the map at the bottom right
legend_control = CustomControl(legend_html, position='bottomright')
m2.add_child(legend_control)    

# Buildings WMS layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",  # workspace-level endpoint
    name="Buildings Toumpa (WMS)",
    layers="project1:buildings_anopoli",  # exactly as it appears in GetCapabilities
    styles="",                           # empty if using the default style
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, # zoom level where the wms layer is visible
).add_to(m2)

# Ano poli WMS area layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",
    name="Area of Toumpa (WMS)",
    layers="project1:area_anopoli",
    styles="",                           
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, 
).add_to(m2)

# Walking Network WMS area layer
folium.WmsTileLayer(
    url="https://thanosgis.me/geoserver/wms",
    name="Walking network (WMS)",
    layers="project1:walking_network_anopoli", 
    styles="",                           
    fmt="image/png",
    transparent=True,
    version="1.1.1",
    maxZoom=20, 
).add_to(m2)

# WFS layer for bus stops
wfs_busa = (
    "https://thanosgis.me/geoserver/wfs?"
    "service=WFS&version=2.0.0&request=GetFeature&"
    "typeName=project1:bus_stops_anopoli&srsName=EPSG:4326&outputFormat=application/json" #specify the crs "srsName=EPSG:4326"
)

# Install those plugins to use Awesome markers
folium.Element(
    '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.awesome-markers/2.0.4/leaflet.awesome-markers.css">'
).add_to(m2)
folium.Element(
    '<script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.awesome-markers/2.0.4/leaflet.awesome-markers.js"></script>'
).add_to(m2)
folium.Element(
    '<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">'
).add_to(m2)

# Create a FeatureGroup for bus stops:
bus_stops_fg2 = folium.FeatureGroup(name="Bus stops (WFS)").add_to(m2)

# Create a MarkerCluster and add it to that FeatureGroup:
cluster2 = MarkerCluster().add_to(bus_stops_fg2)

# Realtime for bus stops
rt2 = Realtime(
    wfs_busa,
    get_feature_id=JsCode("(f) => { return f.properties.stop_id; }"),
    #interval=10000,  # update every 10 seconds
    container=cluster2,  # Add to the cluster
    point_to_layer=point_to_layer_js
)

# Add the FeatureGroup to the map
bus_stops_fg2.add_child(rt2)

# WFS layer for buildings attributes
wfs_buildingsa = (
    "https://thanosgis.me/geoserver/wfs?"
    "service=WFS&version=2.0.0&request=GetFeature&"
    "typeName=project1:buildings_anopoli&srsName=EPSG:4326&outputFormat=application/json" #specify the crs "srsName=EPSG:4326"
)

# Create the Realtime layer to poll the WFS endpoint
rt3 = Realtime(
    wfs_buildingsa,
    get_feature_id=JsCode("(f) => { return f.properties.building_id; }"),
    style=style_js ,      # Apply the transparent style
    on_each_feature=tooltip_js,   # Bind the tooltip
    #interval=10000,                      
)

# Do not add Realtime directly to the map because it adds an extra layer on the layer control
fg3 = folium.FeatureGroup(name="Attributes (WFS)").add_to(m2)
fg3.add_child(rt3)

# Add a fullscreen button and layer control
Fullscreen().add_to(m2)
folium.LayerControl().add_to(m2)

m2

### Save Interactive Maps

In [ ]:
# Ensure the output directory exists
OUTPUT_DIRECTORY.mkdir(parents=True, exist_ok=True)

# Save the maps as html
m.save(OUTPUT_DIRECTORY / "toumpa_interactive.html")
m2.save(OUTPUT_DIRECTORY / "anopoli_interactive.html")